In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as scipy
import seaborn as sns
from pandas import json_normalize
%matplotlib inline
%config IPCompleter.greedy=True


In [ ]:
#Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
archivo='/content/drive/MyDrive/BPA/BUSINESS-TI/content.json'
ruta='/content/drive/MyDrive/BPA/BUSINESS-TI'
df = pd.read_json(archivo)
df

,id_pedido,cliente,fecha,monto,estado,items
0,101,Carlos López,2024-11-18,500,pendiente,"[{'producto': 'Teclado', 'cantidad': 2, 'preci..."
1,102,María García,2024-11-17,750,completado,"[{'producto': 'Silla Gamer', 'cantidad': 1, 'p..."
2,103,Juan Pérez,2024-11-18,120,pendiente,"[{'producto': 'Mouse', 'cantidad': 2, 'precio_..."
3,104,Ana Gómez,2024-11-16,950,completado,"[{'producto': 'Laptop', 'cantidad': 1, 'precio..."
4,105,Luis Martínez,2024-11-19,200,pendiente,"[{'producto': 'Teclado', 'cantidad': 1, 'preci..."
5,106,Sofía Reyes,2024-11-15,300,pendiente,"[{'producto': 'Audífonos', 'cantidad': 2, 'pre..."
6,107,Pedro Morales,2024-11-14,600,completado,"[{'producto': 'Tablet', 'cantidad': 2, 'precio..."
7,108,María García,2024-11-13,450,completado,"[{'producto': 'Monitor', 'cantidad': 3, 'preci..."
8,109,Carlos López,2024-11-12,700,pendiente,"[{'producto': 'Laptop', 'cantidad': 1, 'precio..."
9,110,Ana Gómez,2024-11-11,500,completado,"[{'producto': 'Escritorio', 'cantidad': 1, 'pr..."


# 1. Procesamiento de Pedidos:

a. Filtra los pedidos con estado "pendiente"

In [ ]:
condicion=df["estado"]=="pendiente"
df_pendiente=df[condicion]
df_pendiente

,id_pedido,cliente,fecha,monto,estado,items
0,101,Carlos López,2024-11-18,500,pendiente,"[{'producto': 'Teclado', 'cantidad': 2, 'preci..."
2,103,Juan Pérez,2024-11-18,120,pendiente,"[{'producto': 'Mouse', 'cantidad': 2, 'precio_..."
4,105,Luis Martínez,2024-11-19,200,pendiente,"[{'producto': 'Teclado', 'cantidad': 1, 'preci..."
5,106,Sofía Reyes,2024-11-15,300,pendiente,"[{'producto': 'Audífonos', 'cantidad': 2, 'pre..."
8,109,Carlos López,2024-11-12,700,pendiente,"[{'producto': 'Laptop', 'cantidad': 1, 'precio..."


b. Valida que el campo monto sea igual a la suma de (cantidad *
precio_unitario) de los items. Si no coincide, registra el id_pedido
en un archivo de errores.

In [ ]:
def calcular(item):
  suma=0
  for x in item:
    suma=suma+x["cantidad"]*x["precio_unitario"]
  return suma;

In [ ]:
errores=[]
for _, row in df_pendiente.iterrows():
  if row["monto"]==calcular(row["items"]):
    print(f"""Monto: " {row["monto"]} {calcular(row["items"])}""")
  else:
    errores.append(row["id_pedido"])


Monto: " 500 500.0
Monto: " 120 120.0
Monto: " 200 200.0
Monto: " 300 300.0
Monto: " 700 700.0


In [ ]:
#condicion=df_pendiente["monto"]==df_pendiente["items"].apply(lambda x:sum( item["cantidad"]*item["precio_unitario"]for item in x))

# 2. Generación de Reportes:


a. Genera un archivo CSV con los pedidos pendientes, incluyendo las
columnas: id_pedido, cliente, fecha, monto, y el número total de
productos (items).

In [ ]:
df_pendiente['total_items'] = df_pendiente['items'].apply(lambda x: sum(item['cantidad'] for item in x))
df_pendiente[['id_pedido', 'cliente', 'fecha', 'monto', 'total_items']].to_csv(f'{ruta}/pedidos_pendientes.csv', index=False)


<ipython-input-445-3b9a4961f18c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pendiente['total_items'] = df_pendiente['items'].apply(lambda x: sum(item['cantidad'] for item in x))


b. Calcula el monto total y el promedio de los pedidos pendientes, y agrega
estas métricas al final del archivo CSV como una fila de resumen.


In [ ]:
df_pendiente["monto_total"]=df_pendiente["monto"].sum()
df_pendiente["promedio"]=df_pendiente["monto"].mean()
df_pendiente[['id_pedido', 'cliente', 'fecha', 'monto', 'total_items','monto_total','promedio'
]].to_csv(f'{ruta}/pedidos_pendientes.csv', index=False)

<ipython-input-446-571ce759ae87>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pendiente["monto_total"]=df_pendiente["monto"].sum()
<ipython-input-446-571ce759ae87>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pendiente["promedio"]=df_pendiente["monto"].mean()


# 3. Análisis Avanzado:

a. Identifica el cliente con el mayor monto acumulado en pedidos
completados y genera un archivo JSON con su información, incluyendo el
monto total de sus pedidos.


In [ ]:
df_completados=df[df["estado"]=="completado"]

In [ ]:
cliente_top = df_completados.groupby('cliente')['monto'].sum().idxmax()
monto_total = df_completados.groupby('cliente')['monto'].sum().max()
cliente_mayor_monto={
    "cliente":cliente_top,
    "monto_total":monto_total
}
df_temporal=pd.DataFrame([cliente_mayor_monto])
df_temporal.to_json(f'{ruta}/pedidos_completados.json',orient='records', force_ascii=False)

# 4. Automatización:

a. Diseña un flujo que:
i. Lea el archivo JSON original.
ii. Realice el procesamiento y análisis descrito.
iii. Genere los siguientes archivos:
1. El archivo CSV de pedidos pendientes.
2. El archivo JSON del cliente destacado.
3. Un archivo CSV con los id_pedido con errores
detectados.


In [ ]:
def lectura_de_archivo(ruta):
  return pd.read_json(ruta)

In [ ]:
def procesamiento(df):
  errores=[]
  #separacion de datos
  df_pendiente=df[df["estado"]=="pendiente"]
  df_completado=df[df["estado"]=="completado"]
  #validar los costos
  for _, row in df_pendiente.iterrows():
    if row["monto"]==calcular(row["items"]):
      print(f"""Monto: " {row["monto"]} {calcular(row["items"])}""")
    else:
      errores.append(row["id_pedido"])
  df_pendiente['total_items'] = df_pendiente['items'].apply(lambda x: sum(item['cantidad'] for item in x))
  df_pendiente[['id_pedido', 'cliente', 'fecha', 'monto', 'total_items']].to_csv(f'{ruta}/pedidos_pendientes.csv', index=False)
  #=====================
  df_pendiente["monto_total"]=df_pendiente["monto"].sum()
  df_pendiente["promedio"]=df_pendiente["monto"].mean()
  df_pendiente[['id_pedido', 'cliente', 'fecha', 'monto', 'total_items','monto_total','promedio'
  ]].to_csv(f'{ruta}/pedidos_pendientes.csv', index=False)
  #==========================
  cliente_top = df_completados.groupby('cliente')['monto'].sum().idxmax()
  monto_total = df_completados.groupby('cliente')['monto'].sum().max()
  cliente_mayor_monto={
      "cliente":cliente_top,
      "monto_total":monto_total
  }
  df_temporal=pd.DataFrame([cliente_mayor_monto])
  df_temporal.to_json(f'{ruta}/pedidos_completados.json',orient='records', force_ascii=False)
  if len(errores) >0:
    pd.DataFrame([errores]).to_json(f'{ruta}/errores_id_pedido.csv')


In [ ]:
df=lectura_de_archivo(archivo)
#Procesamiento y exportacion de los archivos CSV,JSON y CSV
procesamiento(df)


Monto: " 500 500.0
Monto: " 120 120.0
Monto: " 200 200.0
Monto: " 300 300.0
Monto: " 700 700.0
0


<ipython-input-458-bd3ec7e503f4>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pendiente['total_items'] = df_pendiente['items'].apply(lambda x: sum(item['cantidad'] for item in x))
<ipython-input-458-bd3ec7e503f4>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pendiente["monto_total"]=df_pendiente["monto"].sum()
<ipython-input-458-bd3ec7e503f4>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


El codigo fue realizado de manera estructurada, sin embargo se considero agruparlos en bloqueas al momento de finalizar para que sea mas legible a su persona, cumple con los requerimientos solicitados y algunas validaciones. De lo cual puede compilarlo en bloques para ver cada proceso realizado, sino puede compilar los bloques de funciones expuestas al final.